In [1]:
from __future__ import division, print_function

import matplotlib.pyplot as plt
import mplhep as hep
import h5py
import numpy as np
import pandas as pd
import seaborn as sns
import math
import sys

In [2]:
plt.style.use(hep.style.ROOT)

In [3]:
PATH = "/eos/home-m/matheus/output_AQAg/" # Caminho comum para todos os arquivos .h5

# Função que abre os arquivos .h5

$M_{x} = \sqrt{s}\sqrt{\xi_{\text{proton}_{1}}\xi_{\text{proton}_{2}}}$ 

onde $\sqrt{s}=13000$ é a energia do centro de massa

$ Y_{x} = \dfrac{1}{2} \; \ln{ \left(\dfrac{\xi_{\text{proton}_{1}}}{\xi_{\text{proton}_{2}}}\right) } $

In [4]:
def open_file( file ):
    df = None
    with h5py.File( file , 'r' ) as f:
        dset = f['dados']
        #print ( 'antes do corte nos xis:', dset.shape )
        #print ( dset[:,:] )
        array = np.array( dset )
        array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
        #array_cut = (array[:,5] <= 2.4) & (array[:,10] <= 2.4) & (array[:,8] >= 40) & (array[:,9] >= 53) & (array[:,4] >= 200)
        DataSet_ = array[array_cut]
        arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet_[:,15] > 0.04) & (DataSet_[:,14] < 0.111) & (DataSet_[:,15] < 0.138)        
        DataSet_ = DataSet_[arrau_cut_xi]
        #DataSet_ = array
        Mx = 13000 * ( np.sqrt( DataSet_[:,14] * DataSet_[:,15] ) )
        Yx = 0.5 * ( np.log( DataSet_[:,15] / DataSet_[:,14] ) )
        Mww_Mxx =  DataSet_[:,0] / Mx 
        Yww_Yx = DataSet_[:,13] - Yx
        DataSet = np.concatenate( ( DataSet_, Mx.reshape(-1,1), Yx.reshape(-1,1), Mww_Mxx.reshape(-1,1), Yww_Yx.reshape(-1,1) ), axis = 1 )        
        #mask = np.any( np.isnan( DataSet ) , axis = 1 )
        #print(DataSet.shape)
        #print( 'Depois do corte nos xis:', DataSet.shape)
        MultiRP = ( DataSet[:,24] == 1 ) & ( DataSet[:,25] == 1 )
        print( 'MultiRP Events :: ', DataSet[ MultiRP ].shape )
        return DataSet[ MultiRP ]
        return DataSet

# Luminosidade Integrada

In [5]:
SingleMuon_Run2016B = 4.55
SingleMuon_Run2016C = 1.59
SingleMuon_Run2016G = 3.65
Luminosidade        = SingleMuon_Run2016B + SingleMuon_Run2016C + SingleMuon_Run2016G 

# Calculando as Eficiências 

In [6]:
import ROOT
pwd_effi = '/eos/home-m/matheus/SWAN_projects/Acoplamento_Quartico_Anomalo/'

Welcome to JupyROOT 6.22/06


In [7]:
# Eficiência dos Prótons para o Signal

def EffiProtonSignal( Dataset ):
    
    f_SF_Prot_POG = ROOT.TFile.Open( pwd_effi + "PreliminaryEfficiencies_July132020_1D2DMultiTrack.root")

    RadDam_45_F_B = f_SF_Prot_POG.Get("Strips/2016/2016B/h45_2016B_RP3_all_2D")
    RadDam_45_N_B = f_SF_Prot_POG.Get("Strips/2016/2016B/h45_2016B_RP2_all_2D")
    RadDam_45_F_C = f_SF_Prot_POG.Get("Strips/2016/2016C/h45_2016C_RP3_all_2D")
    RadDam_45_N_C = f_SF_Prot_POG.Get("Strips/2016/2016C/h45_2016C_RP2_all_2D")
    RadDam_45_F_G = f_SF_Prot_POG.Get("Strips/2016/2016G/h45_2016G_RP3_all_2D")
    RadDam_45_N_G = f_SF_Prot_POG.Get("Strips/2016/2016G/h45_2016G_RP2_all_2D")
    RadDam_56_F_B = f_SF_Prot_POG.Get("Strips/2016/2016B/h56_2016B_RP103_all_2D")
    RadDam_56_N_B = f_SF_Prot_POG.Get("Strips/2016/2016B/h56_2016B_RP102_all_2D")
    RadDam_56_F_C = f_SF_Prot_POG.Get("Strips/2016/2016C/h56_2016C_RP103_all_2D")
    RadDam_56_N_C = f_SF_Prot_POG.Get("Strips/2016/2016C/h56_2016C_RP102_all_2D")
    RadDam_56_F_G = f_SF_Prot_POG.Get("Strips/2016/2016G/h56_2016G_RP103_all_2D")
    RadDam_56_N_G = f_SF_Prot_POG.Get("Strips/2016/2016G/h56_2016G_RP102_all_2D")

    multitrack_45_F_B = f_SF_Prot_POG.Get("Strips/2016/2016B/h45multitrackeff_2016B_avg_RP3")
    multitrack_45_F_C = f_SF_Prot_POG.Get("Strips/2016/2016C/h45multitrackeff_2016C_avg_RP3")
    multitrack_45_F_G = f_SF_Prot_POG.Get("Strips/2016/2016G/h45multitrackeff_2016G_avg_RP3")
    multitrack_45_N_B = f_SF_Prot_POG.Get("Strips/2016/2016B/h45multitrackeff_2016B_avg_RP2")
    multitrack_45_N_C = f_SF_Prot_POG.Get("Strips/2016/2016C/h45multitrackeff_2016C_avg_RP2")
    multitrack_45_N_G = f_SF_Prot_POG.Get("Strips/2016/2016G/h45multitrackeff_2016G_avg_RP2")
    multitrack_56_F_B = f_SF_Prot_POG.Get("Strips/2016/2016B/h56multitrackeff_2016B_avg_RP103")
    multitrack_56_F_C = f_SF_Prot_POG.Get("Strips/2016/2016C/h56multitrackeff_2016C_avg_RP103")
    multitrack_56_F_G = f_SF_Prot_POG.Get("Strips/2016/2016G/h56multitrackeff_2016G_avg_RP103")
    multitrack_56_N_B = f_SF_Prot_POG.Get("Strips/2016/2016B/h56multitrackeff_2016B_avg_RP102")
    multitrack_56_N_C = f_SF_Prot_POG.Get("Strips/2016/2016C/h56multitrackeff_2016C_avg_RP102")
    multitrack_56_N_G = f_SF_Prot_POG.Get("Strips/2016/2016G/h56multitrackeff_2016G_avg_RP102")
    
    
    weigth_multitrack_45_F_B = multitrack_45_F_B.GetBinContent(1);
    weigth_multitrack_45_F_C = multitrack_45_F_C.GetBinContent(1);
    weigth_multitrack_45_F_G = multitrack_45_F_G.GetBinContent(1);
    weigth_multitrack_45_N_B = multitrack_45_N_B.GetBinContent(1);
    weigth_multitrack_45_N_C = multitrack_45_N_C.GetBinContent(1);
    weigth_multitrack_45_N_G = multitrack_45_N_G.GetBinContent(1);
    weigth_multitrack_56_F_B = multitrack_56_F_B.GetBinContent(1);
    weigth_multitrack_56_F_C = multitrack_56_F_C.GetBinContent(1);
    weigth_multitrack_56_F_G = multitrack_56_F_G.GetBinContent(1);
    weigth_multitrack_56_N_B = multitrack_56_N_B.GetBinContent(1);
    weigth_multitrack_56_N_C = multitrack_56_N_C.GetBinContent(1);
    weigth_multitrack_56_N_G = multitrack_56_N_G.GetBinContent(1);
    
    
    w_raddam_45_Multi_B_N = []
    w_raddam_45_Multi_C_N = []
    w_raddam_45_Multi_G_N = []
    w_raddam_45_Multi_B_F = []
    w_raddam_45_Multi_C_F = []
    w_raddam_45_Multi_G_F = []

    for i in range(0, len( Dataset ) ):
        if [ Dataset[:,22] == 0 ][0][i] and [ Dataset[:,14] <= 0.111 ][0][i] and [ Dataset[:,15] >= 0.04 ][0][i] : 
            w_raddam_45_Multi_B_N.append( RadDam_45_N_B.GetBinContent( RadDam_45_N_B.GetXaxis().FindBin(Dataset[:,26][i] ), RadDam_45_N_B.GetYaxis().FindBin( Dataset[:,28][i] ) ) )
            w_raddam_45_Multi_C_N.append( RadDam_45_N_C.GetBinContent( RadDam_45_N_C.GetXaxis().FindBin(Dataset[:,26][i] ), RadDam_45_N_C.GetYaxis().FindBin( Dataset[:,28][i] ) ) )
            w_raddam_45_Multi_G_N.append( RadDam_45_N_G.GetBinContent( RadDam_45_N_G.GetXaxis().FindBin(Dataset[:,26][i] ), RadDam_45_N_G.GetYaxis().FindBin( Dataset[:,28][i] ) ) )
            w_raddam_45_Multi_B_F.append( RadDam_45_F_B.GetBinContent( RadDam_45_F_B.GetXaxis().FindBin(Dataset[:,30][i] ), RadDam_45_F_B.GetYaxis().FindBin( Dataset[:,32][i] ) ) )
            w_raddam_45_Multi_C_F.append( RadDam_45_F_C.GetBinContent( RadDam_45_F_C.GetXaxis().FindBin(Dataset[:,30][i] ), RadDam_45_F_C.GetYaxis().FindBin( Dataset[:,32][i] ) ) )            
            w_raddam_45_Multi_G_F.append( RadDam_45_F_G.GetBinContent( RadDam_45_F_G.GetXaxis().FindBin(Dataset[:,30][i] ), RadDam_45_F_G.GetYaxis().FindBin( Dataset[:,32][i] ) ) )  
    
    
    
    w_raddam_45_Multi = ((4.55/9.79)*np.array(w_raddam_45_Multi_B_N)+(1.59/9.79)*np.array(w_raddam_45_Multi_C_N)+(3.65/9.79)*np.array(w_raddam_45_Multi_G_N))*((4.55/9.79)*np.array(w_raddam_45_Multi_B_F)+(1.59/9.79)*np.array(w_raddam_45_Multi_C_F)+(3.65/9.79)*np.array(w_raddam_45_Multi_G_F) )        

    w_raddam_56_Multi_B_N = []
    w_raddam_56_Multi_C_N = []
    w_raddam_56_Multi_G_N = []
    w_raddam_56_Multi_B_F = []
    w_raddam_56_Multi_C_F = []
    w_raddam_56_Multi_G_F = []

    for i in range(0, len( Dataset ) ):
        if [ Dataset[:,23] == 1 ][0][i] and [ Dataset[:,14] <= 0.111 ][0][i] and [ Dataset[:,15] >= 0.04 ][0][i]: 
            w_raddam_56_Multi_B_N.append( RadDam_56_N_B.GetBinContent( RadDam_56_N_B.GetXaxis().FindBin(Dataset[:,27][i] ), RadDam_56_N_B.GetYaxis().FindBin( Dataset[:,29][i] ) ) )
            w_raddam_56_Multi_C_N.append( RadDam_56_N_C.GetBinContent( RadDam_56_N_C.GetXaxis().FindBin(Dataset[:,27][i] ), RadDam_56_N_C.GetYaxis().FindBin( Dataset[:,29][i] ) ) )
            w_raddam_56_Multi_G_N.append( RadDam_56_N_G.GetBinContent( RadDam_56_N_G.GetXaxis().FindBin(Dataset[:,27][i] ), RadDam_56_N_G.GetYaxis().FindBin( Dataset[:,29][i] ) ) )
            w_raddam_56_Multi_B_F.append( RadDam_56_F_B.GetBinContent( RadDam_56_F_B.GetXaxis().FindBin(Dataset[:,31][i] ), RadDam_56_F_B.GetYaxis().FindBin( Dataset[:,33][i] ) ) )
            w_raddam_56_Multi_C_F.append( RadDam_56_F_C.GetBinContent( RadDam_56_F_C.GetXaxis().FindBin(Dataset[:,31][i] ), RadDam_56_F_C.GetYaxis().FindBin( Dataset[:,33][i] ) ) )            
            w_raddam_56_Multi_G_F.append( RadDam_56_F_G.GetBinContent( RadDam_56_F_G.GetXaxis().FindBin(Dataset[:,31][i] ), RadDam_56_F_G.GetYaxis().FindBin( Dataset[:,33][i] ) ) )  
    
    
    w_raddam_56_Multi = ( (4.55/9.79)*np.array(w_raddam_56_Multi_B_N)+(1.59/9.79)*np.array(w_raddam_56_Multi_C_N)+(3.65/9.79)*np.array(w_raddam_56_Multi_G_N))*((4.55/9.79)*np.array(w_raddam_56_Multi_B_F)+(1.59/9.79)*np.array(w_raddam_56_Multi_C_F)+(3.65/9.79)*np.array(w_raddam_56_Multi_G_F) )        
    
    
    weigth_45_N = weigth_multitrack_45_N_B*(4.55/9.79) + weigth_multitrack_45_N_C*(1.59/9.79) + weigth_multitrack_45_N_G*(3.65/9.79);
    weigth_45_F = weigth_multitrack_45_F_B*(4.55/9.79) + weigth_multitrack_45_F_C*(1.59/9.79) + weigth_multitrack_45_F_G*(3.65/9.79);
    weigth_45 = (weigth_45_N+weigth_45_F)/2;
    weigth_56_N = weigth_multitrack_56_N_B*(4.55/9.79) + weigth_multitrack_56_N_C*(1.59/9.79) + weigth_multitrack_56_N_G*(3.65/9.79);
    weigth_56_F = weigth_multitrack_56_F_B*(4.55/9.79) + weigth_multitrack_56_F_C*(1.59/9.79) + weigth_multitrack_56_F_G*(3.65/9.79);
    weigth_56 = (weigth_56_N+weigth_56_F)/2;
    return w_raddam_45_Multi * weigth_45 * w_raddam_56_Multi * weigth_56

In [8]:
def EfficienciesStudies( DataSet ):
    
    # Muon Efficiency
    f_SF_ID_BC = ROOT.TFile.Open(pwd_effi + "EfficienciesStudies_2016_legacy_rereco_rootfiles_RunBCDEF_SF_ID.root")
    f_SF_ISO_BC = ROOT.TFile.Open(pwd_effi +  "EfficienciesStudies_2016_legacy_rereco_rootfiles_RunBCDEF_SF_ISO.root" )
    f_SF_trg_BC = ROOT.TFile.Open(pwd_effi +  "EfficienciesAndSF_trg_RunBtoF.root" )
    f_SF_ID_G = ROOT.TFile.Open(pwd_effi + "EfficienciesStudies_2016_legacy_rereco_rootfiles_RunGH_SF_ID.root")
    f_SF_ISO_G = ROOT.TFile.Open(pwd_effi + "EfficienciesStudies_2016_legacy_rereco_rootfiles_RunGH_SF_ISO.root")
    f_SF_trg_G = ROOT.TFile.Open(pwd_effi + "EfficienciesAndSF_trg_RunGH.root")
    '''
    h_SF_ID_BC = ROOT.TH2D()
    h_SF_ISO_BC = ROOT.TH2D()
    h_SF_TRG_BC = ROOT.TH2D()
    h_SF_ID_G = ROOT.TH2D()
    h_SF_ISO_G = ROOT.TH2D()
    h_SF_TRG_G = ROOT.TH2D()
    '''
    h_SF_ID_BC = f_SF_ID_BC.Get("NUM_TightID_DEN_genTracks_eta_pt")
    h_SF_ISO_BC = f_SF_ISO_BC.Get("NUM_TightRelIso_DEN_TightIDandIPCut_eta_pt")
    h_SF_TRG_BC = f_SF_trg_BC.Get("IsoMu24_OR_IsoTkMu24_PtEtaBins/abseta_pt_ratio")
    h_SF_ID_G = f_SF_ID_G.Get("NUM_TightID_DEN_genTracks_eta_pt")
    h_SF_ISO_G = f_SF_ISO_G.Get("NUM_TightRelIso_DEN_TightIDandIPCut_eta_pt")
    h_SF_TRG_G = f_SF_trg_G.Get("IsoMu24_OR_IsoTkMu24_PtEtaBins/abseta_pt_ratio")
    '''
    Get_Object_f_SF_ID_BC = f_SF_ID_BC.GetObject( "NUM_TightID_DEN_genTracks_eta_pt", h_SF_ID_BC )
    Get_Object_f_SF_ISO_BC = f_SF_ID_BC.GetObject( "NUM_TightID_DEN_genTracks_eta_pt", h_SF_ISO_BC )
    Get_Object_f_SF_trg_BC = f_SF_ID_BC.GetObject( "NUM_TightID_DEN_genTracks_eta_pt", h_SF_TRG_BC )
    Get_Object_f_SF_ID_G = f_SF_ID_BC.GetObject( "NUM_TightID_DEN_genTracks_eta_pt", h_SF_ID_G )
    Get_Object_f_SF_ISO_G = f_SF_ID_BC.GetObject( "NUM_TightID_DEN_genTracks_eta_pt", h_SF_ISO_G )
    Get_Object_f_SF_trg_G = f_SF_ID_BC.GetObject( "NUM_TightID_DEN_genTracks_eta_pt", h_SF_TRG_G )
    '''
    # Extra Tracks Reweight
    f_track_re = ROOT.TFile.Open( "Extra_track_reweight.root" )
    f_track_re_ele = ROOT.TFile.Open( "Extra_track_reweight_ele.root" )
    #h_track_re = ROOT.TH2D()
    
    #Get_Object_f_track_re = f_SF_ID_BC.GetObject( "NUM_TightID_DEN_genTracks_eta_pt", h_track_re )

    
    h_track_re = f_track_re.Get( "h5" )
    #h_track_re_ele = f_track_re_ele.Get( "h5" )
    
    weigth_id_BC  = [] # For muon 
    weigth_iso_BC = [] # For muon
    weigth_trg_BC = [] # For muon
    weight_id_G = []   # For muon
    weight_iso_G = []  # For muon
    weight_trg_G = []  # For muon
    
    weigth_track_re = [] # For Extra Tracks
    weigth_track_re_ele = [] # For Extra Tracks


    for i in range( 0, len( DataSet ) ):
        if [DataSet[:,9] < 120][0][i]:
            weigth_id_BC.append( h_SF_ID_BC.GetBinContent( h_SF_ID_BC.GetXaxis().FindBin( DataSet[:,10][i] ), h_SF_ISO_BC.GetYaxis().FindBin( DataSet[:,9][i] ) ) )
        else:
            weigth_id_BC.append( h_SF_ID_BC.GetBinContent( h_SF_ID_BC.GetXaxis().FindBin( DataSet[:,10][i] ), h_SF_ISO_BC.GetYaxis().FindBin( 119 ) ) )
       
    
    for i in range( 0, len( DataSet ) ):
        if [ DataSet[:,9] < 120 ][0][i]:
            weigth_iso_BC.append( h_SF_ISO_BC.GetBinContent( h_SF_ISO_BC.GetXaxis().FindBin( DataSet[:,10][i] ), h_SF_ISO_BC.GetYaxis().FindBin( DataSet[:,9][i] ) ) )
        else:
            weigth_iso_BC.append( h_SF_ISO_BC.GetBinContent( h_SF_ISO_BC.GetXaxis().FindBin( DataSet[:,10][i] ), h_SF_ISO_BC.GetYaxis().FindBin( 119 ) ) )
   
    for i in range( 0, len( DataSet ) ):
        if [ DataSet[:,9] < 120 ][0][i]:
            weight_id_G.append( h_SF_ID_G.GetBinContent( h_SF_ID_G.GetXaxis().FindBin( DataSet[:,10][i] ), h_SF_ID_G.GetYaxis().FindBin( DataSet[:,9][i] ) ) )
        else:
            weight_id_G.append( h_SF_ID_G.GetBinContent( h_SF_ID_G.GetXaxis().FindBin( DataSet[:,10][i] ), h_SF_ID_G.GetYaxis().FindBin( 119 ) ) )
    
    for i in range( 0, len( DataSet ) ):
        if [ DataSet[:,9] < 120 ][0][i]:
            weight_iso_G.append( h_SF_ISO_G.GetBinContent( h_SF_ISO_G.GetXaxis().FindBin( DataSet[:,10][i] ), h_SF_ISO_G.GetYaxis().FindBin( DataSet[:,9][i] ) ) )
        else:
            weight_iso_G.append( h_SF_ISO_G.GetBinContent( h_SF_ISO_G.GetXaxis().FindBin( DataSet[:,10][i] ), h_SF_ISO_G.GetYaxis().FindBin( 119 ) ) )

        
    for i in range( 0, len( DataSet ) ):
        if [ DataSet[:,9] < 500 ][0][i]:
            weigth_trg_BC.append( h_SF_TRG_BC.GetBinContent( h_SF_TRG_BC.GetXaxis().FindBin( DataSet[:,10][i] ), h_SF_TRG_BC.GetYaxis().FindBin( DataSet[:,9][i] ) ) )
        else:
            weigth_trg_BC.append( h_SF_TRG_BC.GetBinContent( h_SF_TRG_BC.GetXaxis().FindBin( DataSet[:,10][i] ), h_SF_TRG_BC.GetYaxis().FindBin( 499 ) ) )
    
    for i in range( 0, len( DataSet ) ):
        if [ DataSet[:,9] < 500 ][0][i]:
            weight_trg_G.append( h_SF_TRG_G.GetBinContent( h_SF_TRG_G.GetXaxis().FindBin( DataSet[:,10][i] ), h_SF_TRG_G.GetYaxis().FindBin( DataSet[:,9][i] ) ) )
        else:
            weight_trg_G.append( h_SF_TRG_G.GetBinContent( h_SF_TRG_G.GetXaxis().FindBin( DataSet[:,10][i] ), h_SF_TRG_G.GetYaxis().FindBin( 499 ) ) )
                     
        
    for i in range( 0, len( DataSet ) ):
        if [ DataSet[:,11] == 0 ][0][i]:
            weigth_track_re.append( h_track_re.GetBinContent( h_track_re.GetXaxis().FindBin( 0.1 ) ) )
            #weigth_track_re_ele.append( h_track_re_ele.GetBinContent(h_track_re_ele.GetXaxis().FindBin(0.1) ) )
        else:
            weigth_track_re.append( h_track_re.GetBinContent( h_track_re.GetXaxis().FindBin( DataSet[:,11][i] ) ) )
            #weigth_track_re_ele.append( h_track_re_ele.GetBinContent(h_track_re_ele.GetXaxis().FindBin( DataSet[:,11][i] ) ) )
            
    weigth_id = np.array( weigth_id_BC )*(6.14/9.79) + np.array( weight_id_G )*(3.65/9.79)
    weigth_iso = np.array( weigth_iso_BC )*(6.14/9.79) + np.array( weight_iso_G )*(3.65/9.79)
    weigth_trg = np.array( weigth_trg_BC )*(6.14/9.79) + np.array( weight_trg_G )*(3.65/9.79)
    
    return weigth_id*weigth_iso*weigth_trg*weigth_track_re

# Amostras de Signal

* ## Normalização 

Seção de Choque

In [9]:
cross_section_SM       = 40.41*0.17
cross_section_ANOMALO1 = 166.1*0.17 
cross_section_ANOMALO2 = 41.90*0.17
cross_section_ANOMALO3 = 48.75*0.17
cross_section_ANOMALO4 = 61.14*0.17
cross_section_ANOMALO5 = 41.58*0.17
cross_section_ANOMALO6 = 44.93*0.17
cross_section_ANOMALO7 = 58.18*0.17 
cross_section_ANOMALO8 = 150.3*0.17

Número de Eventos 

In [10]:
number_events_SM       = 35000
number_events_ANOMALO1 = 35000
number_events_ANOMALO2 = 35000
number_events_ANOMALO3 = 35000
number_events_ANOMALO4 = 35000
number_events_ANOMALO5 = 35000
number_events_ANOMALO6 = 35000
number_events_ANOMALO7 = 35000
number_events_ANOMALO8 = 35000

Normalizacao dos eventos de SIGNAL

In [11]:
norm_SM = ( cross_section_SM * Luminosidade ) / ( number_events_SM )
norm_ANOMALO1 = ( cross_section_ANOMALO1 * Luminosidade ) / ( number_events_ANOMALO1 )
norm_ANOMALO2 = ( cross_section_ANOMALO2 * Luminosidade ) / ( number_events_ANOMALO2 )
norm_ANOMALO3 = ( cross_section_ANOMALO3 * Luminosidade ) / ( number_events_ANOMALO3 )
norm_ANOMALO4 = ( cross_section_ANOMALO4 * Luminosidade ) / ( number_events_ANOMALO4 )
norm_ANOMALO5 = ( cross_section_ANOMALO5 * Luminosidade ) / ( number_events_ANOMALO5 )
norm_ANOMALO6 = ( cross_section_ANOMALO6 * Luminosidade ) / ( number_events_ANOMALO6 )
norm_ANOMALO7 = ( cross_section_ANOMALO7 * Luminosidade ) / ( number_events_ANOMALO7 )
norm_ANOMALO8 = ( cross_section_ANOMALO8 * Luminosidade ) / ( number_events_ANOMALO8 )

* ## Importando as amostras de Signal

In [12]:
SM = PATH + 'output-SM.h5'
ANOMALO1 = PATH + 'output-ANOMALO1.h5'
ANOMALO2 = PATH + 'output-ANOMALO2.h5'
ANOMALO3 = PATH + 'output-ANOMALO3.h5'
ANOMALO4 = PATH + 'output-ANOMALO4.h5'
ANOMALO5 = PATH + 'output-ANOMALO5.h5'
ANOMALO6 = PATH + 'output-ANOMALO6.h5'
ANOMALO7 = PATH + 'output-ANOMALO7.h5'
ANOMALO8 = PATH + 'output-ANOMALO8.h5'

In [13]:
SM =       open_file(SM)
ANOMALO1 = open_file(ANOMALO1)
ANOMALO2 = open_file(ANOMALO2)
ANOMALO3 = open_file(ANOMALO3)
ANOMALO4 = open_file(ANOMALO4)
ANOMALO5 = open_file(ANOMALO5)
ANOMALO6 = open_file(ANOMALO6)
ANOMALO7 = open_file(ANOMALO7)
ANOMALO8 = open_file(ANOMALO8)

MultiRP Events ::  (31, 38)
MultiRP Events ::  (386, 38)
MultiRP Events ::  (45, 38)

<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in greater
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in less
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in greater
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet_[:,15] > 0.04) & (DataSet_[:,14] < 0.111) & (DataSet_[:,15] < 0.138)
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in less
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet


MultiRP Events ::  (111, 38)
MultiRP Events ::  (151, 38)
MultiRP Events ::  (48, 38)
MultiRP Events ::  (76, 38)
MultiRP Events ::  (164, 38)
MultiRP Events ::  (401, 38)


## Concatenando os pesos no array dos eventos 

In [14]:
weight_SM = np.array( [ norm_SM ]*len( SM ) ).reshape(-1,1)
SM = np.concatenate( ( SM , weight_SM ) , axis = 1 )

weight_ANOMALO1 = np.array( [ norm_ANOMALO1 ]*len( ANOMALO1 ) ).reshape(-1,1)
ANOMALO1 = np.concatenate( ( ANOMALO1 , weight_ANOMALO1 ) , axis = 1 )

weight_ANOMALO2 = np.array( [ norm_ANOMALO2 ]*len( ANOMALO2 ) ).reshape(-1,1)
ANOMALO2 = np.concatenate( ( ANOMALO2 , weight_ANOMALO2 ) , axis = 1 )

weight_ANOMALO3 = np.array( [ norm_ANOMALO3 ]*len( ANOMALO3 ) ).reshape(-1,1)
ANOMALO3 = np.concatenate( ( ANOMALO3 , weight_ANOMALO3 ) , axis = 1 )

weight_ANOMALO4 = np.array( [ norm_ANOMALO4 ]*len( ANOMALO4 ) ).reshape(-1,1)
ANOMALO4 = np.concatenate( ( ANOMALO4 , weight_ANOMALO4 ) , axis = 1 )

weight_ANOMALO5 = np.array( [ norm_ANOMALO5 ]*len( ANOMALO5 ) ).reshape(-1,1)
ANOMALO5 = np.concatenate( ( ANOMALO5 , weight_ANOMALO5 ) , axis = 1 )

weight_ANOMALO6 = np.array( [ norm_ANOMALO6 ]*len( ANOMALO6 ) ).reshape(-1,1)
ANOMALO6 = np.concatenate( ( ANOMALO6 , weight_ANOMALO6 ) , axis = 1 )

weight_ANOMALO7 = np.array( [ norm_ANOMALO7 ]*len( ANOMALO7 ) ).reshape(-1,1) 
ANOMALO7 = np.concatenate( ( ANOMALO7 , weight_ANOMALO7 ) , axis = 1 )

weight_ANOMALO8 = np.array( [ norm_ANOMALO8 ]*len( ANOMALO8) ).reshape(-1,1)
ANOMALO8 = np.concatenate( ( ANOMALO8 , weight_ANOMALO8 ) , axis = 1 )

# Amostras de Background

O estado final do sinal pode ser imitado por muitos processos do Modelo Padrão. o
processos de background que são considerados na análise são listados aqui

* $\textbf{W+jatos}$: 
é o fundo principal. É produzido por fusão parton-parton com uma produção W ( $qg \rightarrow  W q' $ ou $qq'  \rightarrow W g$) onde o $W$ decai em um par lepton-neutrino e o parton(quark ou gluon) dá origem a um jato


* $\textbf{ttbar}$: 
é o segundo fundo principal, já que o quark top sempre decai em um $W$ mais
um quark adicional (> 99% sendo quark inferior), a produção ttbar dá origem a
um processo com dois $W$. Também é produzido por fusão parton-parton
 ($q \bar{q} \rightarrow t\bar{t}$ ou $gg \rightarrow t\bar{t}$).

* $\textbf{QCD}$: 
é um processo de multiplos jatos produzido por fusão parton-parton. As amostras QCD
são gerados com um filtro para aceitar eventos com pelo menos um múon com $p_{T}> 5$ GeV.


* $\textbf{Drell-Yan}$:
é um estado final de par de leptões produzido em uma fusão $q\bar{q}$. O processo tem
jatos adicionais da fragmentação do próton que podem imitar o sinal.

* $\textbf{Single top}$:
pode ser produzido de 3 maneiras diferentes: no canal t no $qb \rightarrow qt$ espalhamento mediado por um W virtual.  no canal tW por um espalhamento de quark de gluon-bottom ($bg \rightarrow  W t$). E no canal s, quando um espalhamento de quark-antiquark dá origem a um estado final top-anti-bottom ($q \bar{q} \rightarrow \bar{b}t$)

* $\textbf{WW,WZ,ZZ inclusivo}$:
são processos são considerados como pano de fundo, mesmo com seus
pequenas seções de choque porque são os processos do Modelo Padrão mais próximos da
topologia de sinal. Eles também são produzidos por fusão parton-parton $q\bar{q} \rightarrow WW$, $q\bar{q} \rightarrow WZ$, $q\bar{q} \rightarrow ZZ$

* ## Normalização 

Seção de Choque

In [15]:

cross_section_TT = 831.7
cross_section_inclusive_WZ = 10.73
cross_section_inclusive_WW = 49.997
cross_section_inclusive_ZZ = 3.28
cross_section_ST_s_channel = 3.365
cross_section_ST_t_channel_top = 136.02
cross_section_ST_t_channel_antitop = 80.95
cross_section_ST_tW_top= 35.85
cross_section_ST_tW_antitop = 35.85
cross_section_DYJetsToLL_Pt_100To250 = 83.12
cross_section_DYJetsToLL_Pt_250To400 = 3.047
cross_section_DYJetsToLL_Pt_400To650 = 0.3921 
cross_section_DYJetsToLL_Pt_650ToInf = 0.0363
cross_section_QCD_Pt_170to300 = 8654. 
cross_section_QCD_Pt_300to470 = 797.3
cross_section_QCD_Pt_470to600 = 79.0
cross_section_QCD_Pt_600to800 = 25.09
cross_section_QCD_Pt_800to1000 = 4.7
cross_section_QCD_Pt_1000toInf = 1.6
cross_section_WJetsToLNu_Pt_100To250 = 677.82 
cross_section_WJetsToLNu_Pt_250To400 = 24.083
cross_section_WJetsToLNu_Pt_400To600 = 3.0563 
cross_section_WJetsToLNu_Pt_600ToInf = 0.4602

Número de Eventos

In [16]:
number_events_TT = 76915549

number_events_inclusive_WZ = 24311445
number_events_inclusive_WW = 6655400 + 1999200
number_events_inclusive_ZZ = 15061141 + 755866

number_events_ST_s_channel = 1000000
number_events_ST_t_channel_top = 43864048
number_events_ST_t_channel_antitop = 38811017
number_events_ST_tW_top = 6952830
number_events_ST_tW_antitop = 6933094

number_events_DYJetsToLL_Pt_100To250 =  2991815 + 2805972 + 2046961
number_events_DYJetsToLL_Pt_250To400 =  594317 + 590806 + 423976
number_events_DYJetsToLL_Pt_400To650 = 604038 + 589842 + 432056
number_events_DYJetsToLL_Pt_650ToInf = 597526 + 430691

number_events_QCD_Pt_170to300 = 19789673 + 7947159
number_events_QCD_Pt_300to470 = 24605508 + 16462878 + 7937590
number_events_QCD_Pt_470to600 = 9847664 + 5668793 + 3972819
number_events_QCD_Pt_600to800 = 9928218 + 5971175 + 401013
number_events_QCD_Pt_800to1000 = 9966149 + 6011849 + 3962749
number_events_QCD_Pt_1000toInf = 9638102 + 3990117

number_events_WJetsToLNu_Pt_100To250 = 10088599 + 9944879 
number_events_WJetsToLNu_Pt_250To400 = 10021205 + 1001250 + 1000132
number_events_WJetsToLNu_Pt_400To600 = 988234 + 951713
number_events_WJetsToLNu_Pt_600ToInf = 985127 + 989482

Normalizacao dos eventos de BACKGROUND 

In [17]:
norm_TT = ( cross_section_TT * 1000 * Luminosidade ) / number_events_TT

norm_inclusive_WZ = ( cross_section_inclusive_WZ * 1000 * Luminosidade ) / number_events_inclusive_WZ
norm_inclusive_ZZ = ( cross_section_inclusive_ZZ * 1000 * Luminosidade ) / number_events_inclusive_ZZ
norm_inclusive_WW = ( cross_section_inclusive_WW * 1000 * Luminosidade ) / number_events_inclusive_WW

norm_ST_s_channel = ( cross_section_ST_s_channel * 1000 * Luminosidade ) / number_events_ST_s_channel
norm_ST_t_channel_top = ( cross_section_ST_t_channel_top * 1000 * Luminosidade ) / number_events_ST_t_channel_top
norm_ST_t_channel_antitop = ( cross_section_ST_t_channel_antitop * 1000 * Luminosidade) / number_events_ST_t_channel_antitop
norm_ST_tW_antitop = ( cross_section_ST_tW_antitop * Luminosidade * 1000 ) / number_events_ST_tW_antitop
norm_ST_tW_top = ( cross_section_ST_tW_top * 1000 * Luminosidade) / number_events_ST_tW_top
 
norm_QCD_Pt_170to300  = ( cross_section_QCD_Pt_170to300 * 1000  * Luminosidade ) / ( number_events_QCD_Pt_170to300  )
norm_QCD_Pt_300to470  = ( cross_section_QCD_Pt_300to470 * 1000  * Luminosidade ) / ( number_events_QCD_Pt_300to470  )
norm_QCD_Pt_470to600  = ( cross_section_QCD_Pt_470to600 * 1000  * Luminosidade ) / ( number_events_QCD_Pt_470to600  )
norm_QCD_Pt_600to800  = ( cross_section_QCD_Pt_600to800 * 1000  * Luminosidade ) / ( number_events_QCD_Pt_600to800  )
norm_QCD_Pt_800to1000 = ( cross_section_QCD_Pt_800to1000 * 1000 * Luminosidade ) / ( number_events_QCD_Pt_800to1000 )
norm_QCD_Pt_1000toInf = ( cross_section_QCD_Pt_1000toInf * 1000 * Luminosidade ) / ( number_events_QCD_Pt_1000toInf )

norm_DYJetsToLL_Pt_100To250 = ( cross_section_DYJetsToLL_Pt_100To250 * 1000 * Luminosidade ) / ( number_events_DYJetsToLL_Pt_100To250 )
norm_DYJetsToLL_Pt_250To400 = ( cross_section_DYJetsToLL_Pt_250To400 * 1000 * Luminosidade ) / ( number_events_DYJetsToLL_Pt_250To400 )
norm_DYJetsToLL_Pt_400To650 = ( cross_section_DYJetsToLL_Pt_400To650 * 1000 * Luminosidade ) / ( number_events_DYJetsToLL_Pt_400To650 )
norm_DYJetsToLL_Pt_650ToInf = ( cross_section_DYJetsToLL_Pt_650ToInf * 1000 * Luminosidade ) / ( number_events_DYJetsToLL_Pt_650ToInf )

norm_WJetsToLNu_Pt_100To250 = ( cross_section_WJetsToLNu_Pt_100To250 * 1000 * Luminosidade ) / ( number_events_WJetsToLNu_Pt_100To250 )
norm_WJetsToLNu_Pt_250To400 = ( cross_section_WJetsToLNu_Pt_250To400 * 1000 * Luminosidade ) / ( number_events_WJetsToLNu_Pt_250To400 )
norm_WJetsToLNu_Pt_400To600 = ( cross_section_WJetsToLNu_Pt_400To600 * 1000 * Luminosidade ) / ( number_events_WJetsToLNu_Pt_400To600 )
norm_WJetsToLNu_Pt_600ToInf = ( cross_section_WJetsToLNu_Pt_600ToInf * 1000 * Luminosidade ) / ( number_events_WJetsToLNu_Pt_600ToInf )

* ## Importando as amostras de Background

* DRELL YAN

In [18]:
# Pt entre 100 e 250 GeV
DY_100_250_2 = PATH + 'output-DY_100_250_2.h5'
DY_100_250_3 = PATH + 'output-DY_100_250_3.h5'
DY_100_250_4 = PATH + 'output-DY_100_250_4.h5'

DY_100_250 = np.concatenate(  ( open_file( DY_100_250_2 ), 
                                open_file( DY_100_250_3 ), 
                                open_file( DY_100_250_4 ) )  , axis = 0 ) 

weight_DY_100_250 = np.array( [ norm_DYJetsToLL_Pt_100To250 ]*len( DY_100_250 ) ).reshape(-1,1)

DY_100_250 = np.concatenate( ( DY_100_250 , weight_DY_100_250 ) , axis = 1 )

# Miguel, fazer o caso acima para todos os arquivos, tanto back quanto signal

# Pt entre 250 e 400 GeV
DY_250_400_2 = PATH + 'output-DY_250_400_2.h5'
DY_250_400_3 = PATH + 'output-DY_250_400_3.h5'
DY_250_400_4 = PATH + 'output-DY_250_400_4.h5'

DY_250_400 = np.concatenate(  ( open_file( DY_250_400_2 ), 
                                open_file( DY_250_400_3 ), 
                                open_file( DY_250_400_4 ) )  , axis = 0 ) 

weight_DY_250_400 = np.array( [ norm_DYJetsToLL_Pt_250To400 ]*len( DY_250_400 ) ).reshape(-1,1)

DY_250_400 = np.concatenate( ( DY_250_400 , weight_DY_250_400 ) , axis = 1 )

# Pt entre 400 e 650 GeV
DY_400_650_1 = PATH + 'output-DY_400_650_1.h5'
DY_400_650_2 = PATH + 'output-DY_400_650_2.h5'
DY_400_650_3 = PATH + 'output-DY_400_650_3.h5'

DY_400_650 = np.concatenate(  ( open_file( DY_400_650_1 ), 
                                open_file( DY_400_650_2 ), 
                                open_file( DY_400_650_3 ) )  , axis = 0 ) 

weight_DY_400_650 = np.array( [ norm_DYJetsToLL_Pt_400To650 ]*len( DY_400_650 ) ).reshape(-1,1)

DY_400_650 = np.concatenate( ( DY_400_650 , weight_DY_400_650 ) , axis = 1 )

# Pt entre 650 e infinito GeV
DY_650_INF_1 = PATH + 'output-DY_650_INF_1.h5'
DY_650_INF_2 = PATH + 'output-DY_650_INF_2.h5'

DY_650_INF = np.concatenate(  ( open_file( DY_650_INF_1 ), 
                                open_file( DY_650_INF_2 ) )  , axis = 0 ) 

weight_DY_650_INF = np.array( [ norm_DYJetsToLL_Pt_650ToInf ]*len( DY_650_INF ) ).reshape(-1,1)

DY_650_INF = np.concatenate( ( DY_650_INF , weight_DY_650_INF ) , axis = 1 )

<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in greater
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in less
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in greater
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet_[:,15] > 0.04) & (DataSet_[:,14] < 0.111) & (DataSet_[:,15] < 0.138)
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in less
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet

MultiRP Events ::  (459, 30)
MultiRP Events ::  (514, 30)
MultiRP Events ::  (338, 30)
MultiRP Events ::  (4493, 30)
MultiRP Events ::  (4527, 30)
MultiRP Events ::  (3210, 30)
MultiRP Events ::  (10673, 30)
MultiRP Events ::  (10460, 30)
MultiRP Events ::  (7821, 30)
MultiRP Events ::  (11336, 30)
MultiRP Events ::  (8086, 30)


In [19]:
# Concatenando todas as constribuições de Pt para o Drell-Yan
Drell_Yan = np.concatenate( ( DY_100_250 , DY_250_400 , DY_400_650 , DY_650_INF ) , axis = 0  ) 

* QCD

In [20]:
# Pt entre 170 e 300 GeV
QCD_170_300_1 = PATH + 'output-QCD_170_300_1.h5'
QCD_170_300_3 = PATH + 'output-QCD_170_300_3.h5'

QCD_170_300 = np.concatenate(  ( open_file( QCD_170_300_1 ), 
                                 open_file( QCD_170_300_3 ) )  , axis = 0 ) 

weight_QCD_170_300 = np.array( [ norm_QCD_Pt_170to300 ]*len( QCD_170_300 ) ).reshape(-1,1)

QCD_170_300 = np.concatenate( ( QCD_170_300 , weight_QCD_170_300 ) , axis = 1 )

# Pt entre 300 e 470 GeV
QCD_300_470_1 = PATH + 'output-QCD_300_470_1.h5'
QCD_300_470_2 = PATH + 'output-QCD_300_470_2.h5'
QCD_300_470_3 = PATH + 'output-QCD_300_470_3.h5'

QCD_300_470 = np.concatenate(  ( open_file( QCD_300_470_1 ), 
                                 open_file( QCD_300_470_2 ),
                                 open_file( QCD_300_470_3 ) )  , axis = 0 ) 

weight_QCD_300_470 = np.array( [ norm_QCD_Pt_300to470 ]*len( QCD_300_470 ) ).reshape(-1,1)

QCD_300_470 = np.concatenate( ( QCD_300_470 , weight_QCD_300_470 ) , axis = 1 )

# Pt entre 470 e 600 GeV
QCD_470_600_1 = PATH + 'output-QCD_470_600_1.h5'
QCD_470_600_2 = PATH + 'output-QCD_470_600_2.h5'
QCD_470_600_3 = PATH + 'output-QCD_470_600_3.h5'

QCD_470_600 = np.concatenate(  ( open_file( QCD_470_600_1 ), 
                                 open_file( QCD_470_600_2 ),
                                 open_file( QCD_470_600_3 ) )  , axis = 0 ) 

weight_QCD_470_600 = np.array( [ norm_QCD_Pt_470to600 ]*len( QCD_470_600 ) ).reshape(-1,1)

QCD_470_600 = np.concatenate( ( QCD_470_600 , weight_QCD_470_600 ) , axis = 1 )

# Pt entre 600 e 800 GeV
QCD_600_800_1 = PATH + 'output-QCD_600_800_1.h5'
QCD_600_800_2 = PATH + 'output-QCD_600_800_2.h5'
QCD_600_800_3 = PATH + 'output-QCD_600_800_3.h5'

QCD_600_800 = np.concatenate(  ( open_file( QCD_600_800_1 ), 
                                 open_file( QCD_600_800_2 ),
                                 open_file( QCD_600_800_3 ) )  , axis = 0 ) 

weight_QCD_600_800 = np.array( [ norm_QCD_Pt_600to800 ]*len( QCD_600_800 ) ).reshape(-1,1)

QCD_600_800 = np.concatenate( ( QCD_600_800 , weight_QCD_600_800 ) , axis = 1 )

# Pt entre 800 e 1000 GeV
QCD_800_1000_1 = PATH + 'output-QCD_800_1000_1.h5'
QCD_800_1000_2 = PATH + 'output-QCD_800_1000_2.h5'
QCD_800_1000_3 = PATH + 'output-QCD_800_1000_3.h5'

QCD_800_1000 = np.concatenate(  ( open_file( QCD_800_1000_1 ), 
                                  open_file( QCD_800_1000_2 ),
                                  open_file( QCD_800_1000_3 ) )  , axis = 0 ) 

weight_QCD_800_1000 = np.array( [ norm_QCD_Pt_800to1000 ]*len( QCD_800_1000 ) ).reshape(-1,1)

QCD_800_1000 = np.concatenate( ( QCD_800_1000 , weight_QCD_800_1000 ) , axis = 1 )

# Pt entre 1000 e infinito GeV
QCD_1000_inf_1 = PATH +'output-QCD_1000_inf_1.h5'
QCD_1000_inf_2 = PATH +'output-QCD_1000_inf_2.h5'

QCD_1000_inf = np.concatenate(  ( open_file( QCD_1000_inf_1 ), 
                                  open_file( QCD_1000_inf_2 ) )  , axis = 0 ) 

weight_QCD_1000_inf = np.array( [ norm_QCD_Pt_1000toInf ]*len( QCD_1000_inf ) ).reshape(-1,1)

QCD_1000_inf = np.concatenate( ( QCD_1000_inf , weight_QCD_1000_inf ) , axis = 1 )

<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in greater
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in less
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in greater
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet_[:,15] > 0.04) & (DataSet_[:,14] < 0.111) & (DataSet_[:,15] < 0.138)
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in less
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet

MultiRP Events ::  (166, 30)
MultiRP Events ::  (48, 30)
MultiRP Events ::  (1957, 30)
MultiRP Events ::  (1482, 30)
MultiRP Events ::  (608, 30)
MultiRP Events ::  (1709, 30)
MultiRP Events ::  (1024, 30)
MultiRP Events ::  (642, 30)
MultiRP Events ::  (2408, 30)
MultiRP Events ::  (1466, 30)
MultiRP Events ::  (953, 30)
MultiRP Events ::  (3421, 30)
MultiRP Events ::  (1975, 30)
MultiRP Events ::  (1339, 30)
MultiRP Events ::  (4347, 30)
MultiRP Events ::  (1741, 30)


In [21]:
# Concatenando todas as constribuições de Pt para o QCD
QCD = np.concatenate( ( QCD_170_300 , QCD_300_470 , QCD_470_600 , QCD_600_800 , QCD_800_1000 , QCD_1000_inf ) , axis = 0 )

* SINGLE ANTI-TOP

In [22]:
Single_Antitop = PATH + 'output-Single_Antitop.h5'
file_Single_Antitop = open_file(Single_Antitop)

weight_ST_tW_antitop = np.array( [ norm_ST_tW_antitop ]*len( file_Single_Antitop ) ).reshape(-1,1)
Single_Antitop = np.concatenate( ( file_Single_Antitop , weight_ST_tW_antitop ) , axis = 1 )

<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in greater
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in less
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in greater
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet_[:,15] > 0.04) & (DataSet_[:,14] < 0.111) & (DataSet_[:,15] < 0.138)
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in less
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet

MultiRP Events ::  (9473, 30)


* SINGLE ANTI-TOP tCHANEL

In [23]:
Single_Antitop_tChannel = PATH + 'output-Single_Antitop_tChannel.h5'
file_Single_Antitop_tChannel = open_file(Single_Antitop_tChannel)

weight_Antitop_tChannel = np.array( [ norm_ST_t_channel_antitop ]*len( file_Single_Antitop_tChannel ) ).reshape(-1,1)
Single_Antitop_tChannel = np.concatenate( ( file_Single_Antitop_tChannel , weight_Antitop_tChannel ) , axis = 1 )

<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in greater
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in less
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in greater
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet_[:,15] > 0.04) & (DataSet_[:,14] < 0.111) & (DataSet_[:,15] < 0.138)
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in less
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet

MultiRP Events ::  (2105, 30)


* SINGLE TOP

In [24]:
Single_Top = PATH + 'output-Single_Top.h5'
file_Single_Top = open_file(Single_Top)

weight_Single_Top = np.array( [ norm_ST_tW_top ]*len( file_Single_Top ) ).reshape(-1,1)
Single_Top = np.concatenate( ( file_Single_Top , weight_Single_Top ) , axis = 1 )

<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in greater
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in less
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in greater
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet_[:,15] > 0.04) & (DataSet_[:,14] < 0.111) & (DataSet_[:,15] < 0.138)
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in less
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet

MultiRP Events ::  (9819, 30)


* SINGLE TOP sCHANNEL

In [25]:
Single_Top_sChanel = PATH + 'output-Single_Top_sChanel.h5'
file_Single_Top_sChanel = open_file(Single_Top_sChanel)

weight_Single_Top_sChanel = np.array( [ norm_ST_s_channel ]*len( file_Single_Top_sChanel ) ).reshape(-1,1)
Single_Top_sChanel = np.concatenate( ( file_Single_Top_sChanel , weight_Single_Top_sChanel ) , axis = 1 )

<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in greater
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in less
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in greater
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet_[:,15] > 0.04) & (DataSet_[:,14] < 0.111) & (DataSet_[:,15] < 0.138)
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in less
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet

MultiRP Events ::  (289, 30)


* SINGLE TOP tCHANNEL

In [26]:
Single_Top_tChannel = PATH + 'output-Single_Top_tChannel.h5'
file_Single_Top_tChannel = open_file(Single_Top_tChannel)

weight_Single_Top_tChannel = np.array( [ norm_ST_t_channel_top ]*len( file_Single_Top_tChannel ) ).reshape(-1,1)
Single_Top_tChannel = np.concatenate( ( file_Single_Top_tChannel , weight_Single_Top_tChannel ) , axis = 1 )

<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in greater
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:8: RuntimeWarning: invalid value encountered in less
  array_cut = (array[:,0] > 600) & (array[:,1] > 200) & (array[:,2] > 2) & (array[:,3] > 2) & (array[:,4] > 200) & (array[:,5] < 2.4) & (array[:,7] < 0.6) & (array[:,8] > 40) & (array[:,9] > 53)  & (array[:,10] < 2.4) # Corte no xi1 e no xi2
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in greater
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet_[:,15] > 0.04) & (DataSet_[:,14] < 0.111) & (DataSet_[:,15] < 0.138)
<ipython-input-4-e3cbe47864a3>:11: RuntimeWarning: invalid value encountered in less
  arrau_cut_xi = (DataSet_[:,14] > 0.04) & (DataSet

MultiRP Events ::  (2791, 30)


In [27]:
single_top = np.concatenate( ( Single_Top_tChannel, 
                               Single_Top_sChanel , 
                               Single_Top , 
                               Single_Antitop_tChannel , 
                               Single_Antitop) , axis = 0 )

* TTBAR

In [ ]:
TTbar = PATH + 'output-ttbar.h5'
file_TTbar = open_file(TTbar)

weight_Single_Top_sChanel = np.array( [ norm_TT ]*len( file_TTbar ) ).reshape(-1,1)
TTbar = np.concatenate( ( file_TTbar , weight_Single_Top_sChanel ) , axis = 1 )

In [ ]:
TTbar.shape

* W + JETS

In [ ]:
# Pt entre 100 e 250 GeV

WJets_100_250_2 = PATH + 'output-WJets_100_250_2.h5'
WJets_100_250_3 = PATH + 'output-WJets_100_250_3.h5'

WJets_100_250 = np.concatenate(  ( open_file( WJets_100_250_2 ), 
                                   open_file( WJets_100_250_3 ) )  , axis = 0 ) 

weight_WJets_100_250 = np.array( [ norm_WJetsToLNu_Pt_100To250 ]*len( WJets_100_250 ) ).reshape(-1,1)

WJets_100_250 = np.concatenate( ( WJets_100_250 , weight_WJets_100_250 ) , axis = 1 )


# Pt entre 250 e 400 GeV 

WJets_250_400_1 = PATH +'output-WJets_250_400_1.h5'
WJets_250_400_2 = PATH +'output-WJets_250_400_2.h5'
WJets_250_400_3 = PATH +'output-WJets_250_400_3.h5'

WJets_250_400 = np.concatenate(  ( open_file( WJets_250_400_1 ), 
                                   open_file( WJets_250_400_2 ),
                                  (open_file( WJets_250_400_3 ) ))  , axis = 0 ) 

weight_WJets_250_400 = np.array( [ norm_WJetsToLNu_Pt_250To400 ]*len( WJets_250_400 ) ).reshape(-1,1)

WJets_250_400 = np.concatenate( ( WJets_250_400 , weight_WJets_250_400 ) , axis = 1 )

# Pt entre 400 e 600 GeV
WJets_400_600_1 = PATH + 'output-WJets_400_600_1.h5'
WJets_400_600_2 = PATH + 'output-WJets_400_600_2.h5'

WJets_400_600 = np.concatenate(  ( open_file( WJets_400_600_1 ), 
                                   open_file( WJets_400_600_2 ) )  , axis = 0 ) 

weight_WJets_400_600 = np.array( [ norm_WJetsToLNu_Pt_400To600 ]*len( WJets_400_600 ) ).reshape(-1,1)

WJets_400_600 = np.concatenate( ( WJets_400_600 , weight_WJets_400_600 ) , axis = 1 )

# Pt entre 600 e infinito GeV
WJets_600_inf_1 = PATH + 'output-WJets_600_inf_1.h5'
WJets_600_inf_2 = PATH + 'output-WJets_600_inf_2.h5'

WJets_600_inf = np.concatenate(  ( open_file( WJets_600_inf_1 ), 
                                   open_file( WJets_600_inf_2 ) )  , axis = 0 ) 

weight_WJets_600_inf = np.array( [ norm_WJetsToLNu_Pt_600ToInf ]*len( WJets_600_inf ) ).reshape(-1,1)

WJets_600_inf = np.concatenate( ( WJets_600_inf , weight_WJets_600_inf ) , axis = 1 )

In [ ]:
# Concatenando todas as constribuições de Pt para o W + Jets
WJets = np.concatenate( ( WJets_100_250 , WJets_250_400 , WJets_400_600 , WJets_600_inf ) , axis = 0 )

* INCLUSIVO WW

In [ ]:
WW_1 = PATH + 'output-WW1.h5'
WW_2 = PATH + 'output-WW2.h5'

WW = np.concatenate(  ( open_file( WW_1 ), 
                        open_file( WW_2 ) )  , axis = 0 ) 

weight_WW = np.array( [ norm_inclusive_WW ]*len( WW ) ).reshape(-1,1)
WW = np.concatenate( ( WW , weight_WW ) , axis = 1 )

* INCLUSIVO WZ

In [ ]:
WZ = PATH + 'output-WZ.h5'
file_WZ = open_file(WZ)

weight_WZ = np.array( [ norm_inclusive_WZ ]*len( file_WZ ) ).reshape(-1,1)
WZ = np.concatenate( ( file_WZ , weight_WZ ) , axis = 1 )

* INCLUSIVO ZZ

In [ ]:
ZZ_1 = PATH + 'output-ZZ1.h5'
ZZ_2 = PATH + 'output-ZZ2.h5'

ZZ = np.concatenate(  ( open_file( ZZ_1 ), 
                        open_file( ZZ_2 ) )  , axis = 0 ) 

weight_ZZ = np.array( [ norm_inclusive_ZZ ]*len( ZZ ) ).reshape(-1,1)
ZZ = np.concatenate( ( ZZ , weight_ZZ ) , axis = 1 )

In [ ]:
VV_inclusivo = np.concatenate( ( WW , WZ , ZZ ) , axis = 0 )

In [ ]:
VV_inclusivo.shape

# Dados - 2016

In [ ]:
print('Data Era B ','\n')
Data_B = open_file( PATH + 'output-Data_B.h5')
print('\n')

print('Data Era C \n')
Data_C = open_file( PATH + 'output-Data_C.h5')
print('\n')

print('Data Era G \n')
Data_G = open_file( PATH + 'output-Data_G.h5')
print('\n',Data_G.shape)


In [ ]:
Dadinhos_reais = np.concatenate( ( Data_B , Data_C ,  Data_G ) , axis = 0 )
pd.DataFrame( Dadinhos_reais )

In [ ]:
'''label_back = [ r'$t\bar{t}}$',  'W+Jatos','QCD', '(WW, WZ, ZZ) Inclusivo', 'Single-Top', 'Drell-Yan', ]'''

In [ ]:
'''label_signal = [
r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=0.0$ (SM)',
r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=2 \times 10^{-5}$',
r'WWCEP $\alpha_{C}^{W}/\Lambda^{2}=8 \times 10^{-6}$',
r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 0.5 \times 10^{-6} $',
r'WWCEP $\alpha_{0}^{W}/\Lambda^{2} = 5.0 \times 10^{-6} $'
          ]'''

## Data set de Sinal apenas MultiRP

In [ ]:
multiRP_SM = np.concatenate( ( SM, EfficienciesStudies(SM).reshape(-1,1) * EffiProtonSignal(SM).reshape(-1,1) * SM[:,12].reshape(-1,1) * SM[:,38].reshape(-1,1) ), axis = 1 ) 
multiRP_ANOMALO1 = np.concatenate( ( ANOMALO1, EfficienciesStudies(ANOMALO1).reshape(-1,1) * EffiProtonSignal(ANOMALO1).reshape(-1,1) * ANOMALO1[:,12].reshape(-1,1) * ANOMALO1[:,38].reshape(-1,1)), axis = 1 ) 
multiRP_ANOMALO2 = np.concatenate( ( ANOMALO2, EfficienciesStudies(ANOMALO2).reshape(-1,1) * EffiProtonSignal(ANOMALO2).reshape(-1,1) * ANOMALO2[:,12].reshape(-1,1) * ANOMALO2[:,38].reshape(-1,1)), axis = 1 ) 
multiRP_ANOMALO3 = np.concatenate( ( ANOMALO3, EfficienciesStudies(ANOMALO3).reshape(-1,1) * EffiProtonSignal(ANOMALO3).reshape(-1,1) * ANOMALO3[:,12].reshape(-1,1) * ANOMALO3[:,38].reshape(-1,1)), axis = 1 ) 
multiRP_ANOMALO4 = np.concatenate( ( ANOMALO4, EfficienciesStudies(ANOMALO4).reshape(-1,1) * EffiProtonSignal(ANOMALO4).reshape(-1,1) * ANOMALO4[:,12].reshape(-1,1) * ANOMALO4[:,38].reshape(-1,1)), axis = 1 ) 
multiRP_ANOMALO5 = np.concatenate( ( ANOMALO5, EfficienciesStudies(ANOMALO5).reshape(-1,1) * EffiProtonSignal(ANOMALO5).reshape(-1,1) * ANOMALO5[:,12].reshape(-1,1) * ANOMALO5[:,38].reshape(-1,1)), axis = 1 ) 
multiRP_ANOMALO6 = np.concatenate( ( ANOMALO6, EfficienciesStudies(ANOMALO6).reshape(-1,1) * EffiProtonSignal(ANOMALO6).reshape(-1,1) * ANOMALO6[:,12].reshape(-1,1) * ANOMALO6[:,38].reshape(-1,1)), axis = 1 ) 
multiRP_ANOMALO7 = np.concatenate( ( ANOMALO7, EfficienciesStudies(ANOMALO7).reshape(-1,1) * EffiProtonSignal(ANOMALO7).reshape(-1,1) * ANOMALO7[:,12].reshape(-1,1) * ANOMALO7[:,38].reshape(-1,1)), axis = 1 ) 
multiRP_ANOMALO8 = np.concatenate( ( ANOMALO8, EfficienciesStudies(ANOMALO8).reshape(-1,1) * EffiProtonSignal(ANOMALO8).reshape(-1,1) * ANOMALO8[:,12].reshape(-1,1) * ANOMALO8[:,38].reshape(-1,1)), axis = 1 ) 


In [ ]:
columns_Signal = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'PUWeight', 'Yww', 'xi1', 'xi2', 'angulo_X_1','angulo_X_2',
'angulo_Y_1', 'angulo_Y_2', 'rpid_1', 'rpid_2', 'arm1', 'arm2', 'ismultirp_1','ismultirp_2', 'near_x1','near_x1',
'near_y1', 'near_y2', 'far_x1','far_x1','far_y1', 'far_y2','Mx', 'Yx', 'Mww/Mx', 'Yww_Yx', 'Norm','weight']

In [ ]:
DataFrame_multiRP_SM = pd.DataFrame( multiRP_SM, columns = columns_Signal )
DataFrame_multiRP_ANOMALO1 = pd.DataFrame( multiRP_ANOMALO1, columns = columns_Signal )
DataFrame_multiRP_ANOMALO2 = pd.DataFrame( multiRP_ANOMALO2, columns = columns_Signal )
DataFrame_multiRP_ANOMALO3 = pd.DataFrame( multiRP_ANOMALO3, columns = columns_Signal )
DataFrame_multiRP_ANOMALO4 = pd.DataFrame( multiRP_ANOMALO4, columns = columns_Signal )
DataFrame_multiRP_ANOMALO5 = pd.DataFrame( multiRP_ANOMALO5, columns = columns_Signal )
DataFrame_multiRP_ANOMALO6 = pd.DataFrame( multiRP_ANOMALO6, columns = columns_Signal )
DataFrame_multiRP_ANOMALO7 = pd.DataFrame( multiRP_ANOMALO7, columns = columns_Signal )
DataFrame_multiRP_ANOMALO8 = pd.DataFrame( multiRP_ANOMALO8, columns = columns_Signal )

## Data set de Background apenas MultiRP

In [ ]:
multiRP_Drell_Yan = np.concatenate(  (Drell_Yan ,  Drell_Yan[:,30].reshape(-1,1) * Drell_Yan[:,12].reshape(-1,1)  ), axis = 1 )

In [ ]:
multiRP_QCD = np.concatenate(  (QCD ,  QCD[:,30].reshape(-1,1) * QCD[:,12].reshape(-1,1)  ), axis = 1 )

In [ ]:
multiRP_single_top = np.concatenate(  (single_top ,  single_top[:,30].reshape(-1,1) * single_top[:,12].reshape(-1,1)  ), axis = 1 )

In [ ]:
multiRP_TTbar = np.concatenate(  (TTbar , TTbar[:,30].reshape(-1,1) * TTbar[:,12].reshape(-1,1)  ), axis = 1 )

In [ ]:
multiRP_WJets = np.concatenate(  (WJets , WJets[:,30].reshape(-1,1) * WJets[:,12].reshape(-1,1) ), axis = 1 )

In [ ]:
multiRP_VV_inclusivo = np.concatenate(  (VV_inclusivo , VV_inclusivo[:,30].reshape(-1,1) * VV_inclusivo[:,12].reshape(-1,1)  ), axis = 1 )

In [ ]:
columns_MC = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'PUWeight', 'Yww', 'xi1', 'xi2', 'angulo_X_1','angulo_X_2',
'angulo_Y_1', 'angulo_Y_2', 'rpid_1', 'rpid_2', 'arm1', 'arm2', 'ismultirp_1','ismultirp_2','Mx', 'Yx', 
'Mww/Mx', 'Yww_Yx', 'Norm','weight']

In [ ]:
DataFrame_multiRP_WJets = pd.DataFrame( multiRP_WJets, columns = columns_MC )
DataFrame_multiRP_Drell_Yan = pd.DataFrame( multiRP_Drell_Yan, columns = columns_MC )
DataFrame_multiRP_QCD = pd.DataFrame( multiRP_QCD, columns = columns_MC )
DataFrame_multiRP_single_top = pd.DataFrame( multiRP_single_top, columns = columns_MC )
DataFrame_multiRP_TTbar = pd.DataFrame( multiRP_TTbar, columns = columns_MC )
DataFrame_multiRP_VV_inclusivo = pd.DataFrame( multiRP_VV_inclusivo, columns = columns_MC )

In [ ]:
data_set_back_multirp = pd.concat( [ DataFrame_multiRP_WJets, DataFrame_multiRP_Drell_Yan, DataFrame_multiRP_QCD, 
                                          DataFrame_multiRP_single_top, DataFrame_multiRP_TTbar, DataFrame_multiRP_VV_inclusivo ], axis = 0 )

## Data set de dados contendo apenas MultiRP

In [ ]:
columns_Data = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'PUWeight' ,'Yww', 'xi1', 'xi2', 'angulo_X_1','angulo_X_2',
'angulo_Y_1', 'angulo_Y_2', 'rpid_1', 'rpid_2', 'arm1', 'arm2', 'ismultirp_1','ismultirp_2','Mx', 'Yx', 
'Mww/Mx', 'Yww_Yx', ]

In [ ]:
data_set_dados_multirp = pd.DataFrame( Dadinhos_reais, columns = columns_Data )

## Criando arquivos únicos em .h5 de signal, background e dados

In [ ]:
def salve_dataset( df, label ):
    with h5py.File( label + '.h5', 'w') as f:
        dset = f.create_dataset( 'dados', data = df )
    return dset   

In [ ]:
select_columns_MC = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'PUWeight', 'Yww', 'xi1', 'xi2', 'Mx', 'Yx', 
'Mww/Mx', 'Yww_Yx', 'weight']
select_columns_Data = ['Mww', 'Pt_W_lep', 'dPhi_Whad_Wlep', 'dPhi_jatos_MET', 'jetAK8_pt','jetAK8_eta', 'jetAK8_prunedMass',
'jetAK8_tau21', 'METPt', 'muon_pt', 'muon_eta', 'ExtraTracks', 'PUWeight', 'Yww', 'xi1', 'xi2','Mx', 'Yx', 
'Mww/Mx', 'Yww_Yx']

In [ ]:
#data_set_back_multirp[select_columns_MC].columns

In [ ]:
#salve_dataset(DataFrame_data_set_back_multirp[select_columns_MC] , 'DataSet_back_multiRP')

salve_dataset(data_set_dados_multirp[select_columns_Data], pwd_effi + 'DataSet_dados_multiRP_AllCuts')

salve_dataset(DataFrame_multiRP_Drell_Yan[select_columns_MC], pwd_effi + 'DataSet_multiRP_DrellYan_AllCuts')
salve_dataset(DataFrame_multiRP_QCD[select_columns_MC], pwd_effi + 'DataSet_multiRP_QCD_AllCuts')
salve_dataset(DataFrame_multiRP_single_top[select_columns_MC], pwd_effi + 'DataSet_multiRP_single_top_AllCuts')
salve_dataset(DataFrame_multiRP_TTbar[select_columns_MC], pwd_effi + 'DataSet_multiRP_TTbar_AllCuts')
salve_dataset(DataFrame_multiRP_WJets[select_columns_MC], pwd_effi + 'DataSet_multiRP_WJets_AllCuts')
salve_dataset(DataFrame_multiRP_VV_inclusivo[select_columns_MC], pwd_effi + 'DataSet_multiRP_VV_inclusivo_AllCuts
salve_dataset(DataFrame_multiRP_SM[select_columns_MC], pwd_effi + 'DataSet_SM_multiRP_AllCuts')
salve_dataset(DataFrame_multiRP_ANOMALO1[select_columns_MC], pwd_effi + 'DataSet_ANOMALO1_multiRP_AllCuts')
salve_dataset(DataFrame_multiRP_ANOMALO2[select_columns_MC],pwd_effi +  'DataSet_ANOMALO2_multiRP_AllCuts')
salve_dataset(DataFrame_multiRP_ANOMALO3[select_columns_MC], pwd_effi + 'DataSet_ANOMALO3_multiRP_AllCuts')
salve_dataset(DataFrame_multiRP_ANOMALO4[select_columns_MC], pwd_effi + 'DataSet_ANOMALO4_multiRP_AllCuts')
salve_dataset(DataFrame_multiRP_ANOMALO5[select_columns_MC],pwd_effi +  'DataSet_ANOMALO5_multiRP_AllCuts')
salve_dataset(DataFrame_multiRP_ANOMALO6[select_columns_MC],pwd_effi +  'DataSet_ANOMALO6_multiRP_AllCuts')
salve_dataset(DataFrame_multiRP_ANOMALO7[select_columns_MC],pwd_effi +  'DataSet_ANOMALO7_multiRP_AllCuts')
salve_dataset(DataFrame_multiRP_ANOMALO8[select_columns_MC],pwd_effi +  'DataSet_ANOMALO8_multiRP_AllCuts')

#salve_dataset(data_set_back_multirp[select_columns_MC], pwd_effi + 'DataSet_back_multiRP')
